In [1]:
import os

os.chdir("..")
os.getcwd()

'd:\\Shaukat ali khan\\programming\\AI-Projects\\langchain-projects\\quiz-generator'

### Generating Quiz

In [2]:
QUIZ_GENERATOR_PROMPT = """
You will receive the contents of a particular chapter of a student's book. Your job is to create {no_questions} multiple choice question answers with {difficulty_level} difficulty using those contents and provide them in the form of python list where:
1- All are questions are inside that list.
2- Each question is a dictionary of values.
3- The keys of the question dictionary should be:
question (representing question as str)
choices (list of choices representing each choice as str)
correct_choice (the index of correct choice i.e. the index of correct choice form the choices list.)

Note: Strictly follow the rules and don't provide anything else than the asked. Only provide a clean list ready to process further.

contents: {book_content}
"""

In [34]:
import pandas as pd
import ast
from notebooks.modular_code import Tokenizer
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_codebase.codebase import *



def load_ollama_model(model: str = 'gemma2:2b'):
    return OllamaLLM(model=model)

def get_response_from_model(llm, prompt):
    return llm.invoke(prompt)

def get_quiz_generator_prompt():
    prompt = PromptTemplate(
        input_variables = ['no_questions', 'difficulty_level', 'book_content'],
        template = QUIZ_GENERATOR_PROMPT
    )
    return prompt


def load_dataframe(df_filepath: str):
    if is_valid_csv_path(df_filepath):
        df = pd.read_csv(df_filepath)
        return df
    
def is_valid_csv_path(path: str):
    if is_valid_path(path):
        if path[-3:] == 'csv':
            return True
        else:
            raise Exception(f"Not a valid csv file: {path}")
        
def get_provided_unit_contents(df: pd.DataFrame, class_no: int, subject: str, unit: int):
    """Requires class and unit. Loads df ( will use cached using streamlit in future ) and returns the unit pages as list."""
    required_df = df[(df['class'] == class_no) & (df['subject'] == subject) & (df['unit_no'] == unit)]
    required_content = list(required_df['content'])
    return required_content

def evaluate_generated_questions(response: str):
    """Removes keyword "python" and the backticks from response and then evaluates to a python object e.g. list, dict etc. and returns that."""
    response = response.replace("python", '')
    response = response.replace("```", '')
    response = ast.literal_eval(response)
    return response



tokenizer = Tokenizer()

In [4]:
prompt = get_quiz_generator_prompt()
llm = load_ollama_model()
df = load_dataframe(r"books_df_csvs\Physics 9.csv")

In [ ]:
content = get_provided_unit_contents(df, 9, 3)
content

In [22]:
len(content)

29

In [25]:
content_str = " ".join(content)


In [26]:
content_token_count = tokenizer.get_content_token_count(content_str)
content_token_count

13407